You are a real astronomer (not a theory guy like me) and are planning an expensive observational campaign. You want to know how likely it is that tomorrow night will be clear given the weather tonight (clear or cloudy). The key piece of information here is that the weather tomorrow depends on the weather today. From past history, you know that:

$p(clear \ tomorrow| \ cloudy \ today) = 0.5$

which means that $p(cloudy \ today| \ cloudy \ tomorrow) = 0.5$

We also have $p(cloudy \ tomorrow| \ clear \ today) = 0.1$

which means that $p(clear \ tomorrow| \ clear \ today) = 0.5$

- We can start with the sky conditions today and make predictions going forward more and more into the future.
- This will look like a big decision tree.
- After enough days, we'll reach equilibrium probabilities that have to do with the mean weather statistics (ignoring seasons) and we'll arrive at $p(clear) = 0.83$ and $p(cloudy) = 0.17$.

You get the same answer for day $N$ as day $N + 1$ and it doesn't matter whether it was clear or cloudy on the day that you started. The steps that we have taken in this process are, indeed, a MARKOV CHAIN.

**Tasks**:
- Start off on a cloud day.
- Implement your weather forecast based on the above probabilities
- Run your simulator for N days (with $N > 10^4$)
- Prepare a plot with the number of days on the x axis and the cumulative fraction of, say, sunny days over the number of days so far on the y axis. This is called a trace-plot, showing how our estimate of $p(clear)$ evolves as the chain samples.
- Prepare a histogram of the above plot. This reveals the distribution of $p(clear)$ .
- Use a summary statistics to determine the most likely value and an error on our estimate.

**Important**:
- In MCMC the process must be stationary which basically means that the chain statistics look the same no matter which chunk you look at, e.g. first half, second half, or every other point, etc.
- Obviously that isn't going to be the case in the early steps of the chain. In our example above, after some time the process was stationary, but not in the first few days.
- So, there is a burn-in phase that needs to be discarded. How one determines the number of early steps to discard as burn-in is tricky, but you should always start with a traceplot of your samples!

**Tasks**
- In the above example, experiment with chopping off different numbers of initial points as burn-in.

In [55]:
import numpy as np
import matplotlib.pyplot as plt

In [82]:
def prevision(x, today, p_cloudy_from_cloudy, p_cloudy_from_clear):
    if today == 0: #today cloudy
        if x < p_cloudy_from_cloudy:
            return 0
        else: 
            return 1
    if today == 1: #today clear
        if x < p_cloudy_from_clear:
            return 0
        else: 
            return 1

In [60]:
# Define probality
p_cloudy = 0.17
p_clear = 0.83

p_clear_from_cloudy = 0.5
p_cloudy_from_cloudy = 0.5
p_cloudy_from_clear = 0.1
p_clear_from_clear = 0.9

cloudy = 0
clear = 1

# Number of iterations
N = int(1e4)

In [81]:
# Define starting condition
today = cloudy

# Inizialize the Markov Chain
T = []
T.append(today)

for i in range(1, N+1):
    #print(i, T[i-1])
    x = np.random.uniform(0, 1)
    #print(x)
    tomorrow = prevision(x, T[i-1], p_cloudy_from_cloudy, p_cloudy_from_clear)
    #print(tomorrow)
    T.append(tomorrow)
    #print(T)

print("Cloudy day: " + str(T.count(0)))
print("Clear day: " + str(T.count(1)))

print(f"Probability of cloudy day: {T.count(0) / len(T):.3f}" )
print(f"Probability of clear day: {T.count(1) / len(T):.3f}")

Cloudy day: 1666
Clear day: 8335
Probability of cloudy day: 0.167
Probability of clear day: 0.833
